$$
  \tag{1}
  C_p(T) = a_0 + a_1 T + a_2 \frac{1}{T^2}
$$

#### References
- Timothy K. Risch. "Curve Fits of the NIST-JANAF
Thermochemical Tables." NASA/TM—20205003319/VOL1. NASA. Feb 2021. https://ntrs.nasa.gov/api/citations/20205003319/downloads/H3289FinalVol%201.pdf



In [17]:
from dataclasses import dataclass
from enum import Enum

class JanafReferenceState(Enum):
    SOLID = "solid"
    LIQUID = "liquid"
    GAS = "gas"

@dataclass(frozen=True)
class JanafState:
    ref: JanafReferenceState
    charge: int  # 0, +1, -1

@dataclass(frozen=True)
class JanafEntry:
    dataset: str  # filepath (relative to data_root)


In [ ]:
# eq 1
def specific_heat_eqn(T, a0, a1, a2):
    """
    T: temperature (T)
    a0, a1, a2: fitting parameters

    returns:
    C_p: specific heat, cal/mol-K
    """
    C_p = a0 + a1 * T + a2 / T**2
    return C_p



# Janaf Files



In [ ]:
# https://janaf.nist.gov/tables/Si-002.txt
RAW = r"""Silicon (Si)	Si1(cr)
T(K)	Cp	S	-[G-H(Tr)]/T	H-H(Tr)	delta-f H	delta-f G	log Kf
0	0.	0.	INFINITE	-3.218	0.	0.	0.
100	7.268	3.833	33.351	-2.952	0.	0.	0.
200	15.636	11.665	20.531	-1.773	0.	0.	0.
250	18.221	15.446	19.138	-0.923	0.	0.	0.
298.15	20.000	18.820	18.820	0.	0.	0.	0.
300	20.050	18.943	18.820	0.037	0.	0.	0.
350	21.276	22.132	19.069	1.072	0.	0.	0.
400	22.142	25.032	19.636	2.159	0.	0.	0.
450	22.803	27.680	20.385	3.283	0.	0.	0.
500	23.330	30.110	21.237	4.436	0.	0.	0.
600	24.154	34.440	23.086	6.812	0.	0.	0.
700	24.803	38.212	24.983	9.260	0.	0.	0.
800	25.359	41.562	26.850	11.769	0.	0.	0.
900	25.874	44.579	28.655	14.331	0.	0.	0.
1000	26.338	47.329	30.387	16.942	0.	0.	0.
1100	26.778	49.860	32.044	19.598	0.	0.	0.
1200	27.196	52.208	33.627	22.297	0.	0.	0.
1300	27.614	54.401	35.142	25.037	0.	0.	0.
1400	28.033	56.463	36.592	27.820	0.	0.	0.
1500	28.451	58.411	37.982	30.644	0.	0.	0.
1600	28.870	60.261	39.317	33.510	0.	0.	0.
1685.000	29.225	61.765	40.412	35.979	CRYSTAL <--> LIQUID
1700	29.288	62.024	40.602	36.418	50.177 0.447  0.014
1800	29.706	63.710	41.839	39.368	-49.947	3.418	-0.099
1900	30.125	65.327	43.033	42.359	-49.675	6.376	-0.175
2000	30.543	66.883	44.187	45.393	-49.361	9.318	-0.243
2100	30.962	68.383	45.303	48.468	-49.006	12.243	-0.305
2200	31.380	69.833	46.386	51.585	-48.608	15.151	-0.360
2300	31.798	71.237	47.436	54.744	-48.169	18.039	-0.410
2400	32.217	72.600	48.456	57.945	-47.688	20.908	-0.455
2500	32.635	73.923	49.448	61.187	-47.165	23.755	-0.496"""



In [ ]:
import numpy as np
import pandas as pd

class JanafFile():
  janaf_col_names = [
    "T",
    "Cp",
    "S",
    "phi",
    "H_rel",
    "delta_f_H",
    "delta_f_G",
    "log_Kf"]
  
  def __init__(self):
    self.element_name: str = ""
    self.element_symbol: str = ""
    self.species_id: str = ""
    self.df: pd.DataFrame | None = None

  def process_line_0(self, tokens):
    self.element_name = tokens[0] if len(tokens) > 0 else ""
    self.element_symbol = tokens[1] if len(tokens) > 1 else ""
    self.species_id = tokens[2] if len(tokens) > 2 else ""

  def _try_float(self, tok: str):
    t = tok.strip()
    if t.upper() in {"INFINITE", "INFINITY", "INF"}:
      return np.inf
    t = t.replace("D", "E").replace("d", "E")  # Fortran exponent support
    try:
      return float(t)
    except ValueError:
      return None

  def process_line(self, tokens):
    ncols = len(self.janaf_col_names)
    nums: list[float] = []
    note_tokens: list[str] = []

    for tok in tokens:
      if len(nums) < ncols:
        v = self._try_float(tok)
        if v is None:
          note_tokens.append(tok)
        else:
          nums.append(v)
      else:
        note_tokens.append(tok)

    # pad to fixed width so df columns always match
    if len(nums) < ncols:
      nums.extend([np.nan] * (8 - len(nums)))
    elif len(nums) > ncols:
      # should be rare; keep provenance in notes and trim
      note_tokens.append(f"[extra_numeric={nums[8:]}]")
      nums = nums[:ncols]

    return nums, " ".join(note_tokens).strip()

  def parse_text(self,text: str):
    lines = [k.strip() for k in text.splitlines() if k.strip()]
    rows = []
    notes = []
    raw = []

    for i, line in enumerate(lines):
      tokens = line.split()

      if i == 0:
        self.process_line_0(tokens)
      elif i == 1:
        continue
      else:
        numbers, note = self.process_line(tokens)
        rows.append(numbers)
        notes.append(" ".join(note))
        raw.append(line)
    self.df = pd.DataFrame(
        data=rows, 
        columns=self.janaf_col_names
        )
    self.df["notes"] = notes
    self.df["raw"] = raw
    
  def parse_file(self, path):
    with open(path, "r") as f:
      self.parse_text(text=f.read())


In [ ]:
import os
from typing import Dict
def load_janaf_tables(
    registry: Dict[JanafState, JanafEntry], 
    data_root: str = "."
) -> Dict[JanafState, JanafFile]:
  """
  Load JANAF tables defined by a registry mapping state -> filename.

  Parameters
  ----------
  registry:
      Dict mapping JanafState(ref, charge) to a filename (relative to data_root).
  data_root:
      Base folder containing the files. If None, uses this module's folder.
  verbose:
      If True, prints what was loaded.

  Returns
  -------
  Dict[JanafState, JanafFile]
      Parsed tables keyed by state.
  """
  tables: Dict[JanafState, JanafFile] = {}

  for state, entry in registry.items():
    path = os.path.join(data_root, entry.dataset)
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Missing JANAF file: {path}")

    jf = JanafFile()
    jf.parse_file(path)  # NOTE: pass the resolved path
    tables[state] = jf

    print(f"Loaded {state.ref.value}, charge={state.charge}: {path}")

  return tables

JANAF_AL: dict[JanafState, JanafEntry] = {
    JanafState(JanafReferenceState.SOLID, 0): JanafEntry("janafnist_Al_cr.txt"),
    JanafState(JanafReferenceState.LIQUID, 0): JanafEntry("janafnist_Al_l.txt"),
    JanafState(JanafReferenceState.GAS, 0): JanafEntry("janafnist_Al_g.txt"),
    JanafState(JanafReferenceState.GAS, +1): JanafEntry("janafnist_Al_g_ion_p.txt"),
    JanafState(JanafReferenceState.GAS, -1): JanafEntry("janafnist_Al_g_ion_n.txt"),
}

tables_Al = load_janaf_tables(JANAF_AL, data_root=".")

Loaded solid, charge=0: ./janafnist_Al_cr.txt
Loaded liquid, charge=0: ./janafnist_Al_l.txt
Loaded gas, charge=0: ./janafnist_Al_g.txt
Loaded gas, charge=1: ./janafnist_Al_g_ion_p.txt
Loaded gas, charge=-1: ./janafnist_Al_g_ion_n.txt


In [ ]:
import re
import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import Dict, Callable, Tuple, Optional

# ----------------------------
# 1) Parse JANAF-like pasted text
# ----------------------------

def parse_janaf_text(raw: str) -> pd.DataFrame:
    """
    Parse a JANAF-style table pasted as text.

    Expected columns (whitespace separated):
    T(K) Cp S -[G-H(Tr)]/T H-H(Tr) [optional: delta-f H, delta-f G, log Kf]
    Lines with transition markers are kept as annotations and not numeric rows.

    Returns a dataframe with numeric rows only + a 'note' column if marker text was on the line.
    """
    rows = []
    for line in raw.splitlines():
        line = line.strip()
        if not line:
            continue

        # skip headers
        if line.startswith("Silicon") or line.startswith("T(K)") or line.startswith("Si1"):
            continue

        # detect marker lines like:
        # 1685.000 ... CRYSTAL <--> LIQUID
        # 3504.616 ... LIQUID <--> IDEAL GAS
        # 1685.000 ... TRANSITION
        m = re.match(r"^([0-9]+(?:\.[0-9]+)?)\s+(.*)$", line)
        if not m:
            continue

        T_str = m.group(1)
        rest = m.group(2)

        # numeric tokens at start of rest
        tokens = rest.split()
        numeric = []
        note_parts = []
        for tok in tokens:
            # accept floats like INFINITE? no -> treat as note
            try:
                numeric.append(float(tok))
            except Exception:
                note_parts.append(tok)

        # We need at least: Cp, S, phi, Hrel OR a pure marker row.
        # If row has < 4 numeric values -> treat as marker note only (ignore)
        if len(numeric) < 4:
            continue

        T = float(T_str)
        Cp, S, phi, Hrel = numeric[:4]

        note = " ".join(note_parts) if note_parts else ""

        rows.append(
            {
                "T": T,
                "Cp": Cp,
                "S": S,
                "phi": phi,     # phi := -[G-H(Tr)]/T
                "Hrel": Hrel,   # H-H(Tr)
                "note": note
            }
        )

    df = pd.DataFrame(rows).sort_values("T").reset_index(drop=True)
    return df


# ----------------------------
# 2) Split into phases using known transition temperatures
# ----------------------------

def split_phases_si(df: pd.DataFrame,
                    Tm: float = 1685.0,
                    Tb: float = 3504.616) -> Dict[str, pd.DataFrame]:
    """
    Split Si data into:
      - 'solid'  : T <  Tm (includes 1685- solid-side row if present)
      - 'liquid' : Tm <= T < Tb (includes the liquid row at 1685 if present)
      - 'gas'    : T >= Tb
    For duplicated temperatures (at transition), we keep BOTH in the raw df,
    but phase split here is done by value ranges.
    """
    solid = df[df["T"] < Tm].copy()
    liquid = df[(df["T"] >= Tm) & (df["T"] < Tb)].copy()
    gas = df[df["T"] >= Tb].copy()
    return {"solid": solid, "liquid": liquid, "gas": gas}


# ----------------------------
# 3) Interpolators (linear; stable and transparent)
# ----------------------------

def make_linear_interp(x: np.ndarray, y: np.ndarray) -> Callable[[np.ndarray], np.ndarray]:
    """
    Simple linear interpolator with bounds checking.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    if np.any(np.diff(x) <= 0):
        raise ValueError("x must be strictly increasing for interpolation.")

    def f(xq):
        xq = np.asarray(xq, dtype=float)
        if np.any((xq < x[0]) | (xq > x[-1])):
            raise ValueError(f"Query out of bounds: [{xq.min()}, {xq.max()}] not within [{x[0]}, {x[-1]}].")
        return np.interp(xq, x, y)

    return f


@dataclass
class JanafPhase:
    name: str
    Tmin: float
    Tmax: float
    Cp: Callable[[np.ndarray], np.ndarray]
    S: Callable[[np.ndarray], np.ndarray]
    phi: Callable[[np.ndarray], np.ndarray]   # phi(T) = -[G-H(Tr)]/T
    Hrel: Callable[[np.ndarray], np.ndarray]  # H(T)-H(Tr)

    def G(self, T: np.ndarray, H_Tr: float = 0.0) -> np.ndarray:
        """
        Gibbs free energy in SAME energy units as H_Tr.
        If table's Hrel is in kJ/mol and phi is in J/mol-K (often),
        you MUST unit-consistent. Your pasted table looks like:
          Cp, S, phi in J/mol-K and Hrel in kJ/mol.
        We'll return G in kJ/mol by default by converting phi*T (J/mol) -> kJ/mol.
        """
        T = np.asarray(T, dtype=float)
        phiT_kJ = (T * self.phi(T)) / 1000.0
        return H_Tr - phiT_kJ


def build_phase_object(phase_name: str, df_phase: pd.DataFrame) -> JanafPhase:
    """
    Build a phase object with linear interpolators.
    Assumes:
      Cp, S, phi are J/mol-K
      Hrel is kJ/mol
    """
    d = df_phase.sort_values("T").drop_duplicates(subset=["T"], keep="last")
    T = d["T"].to_numpy()
    Cp = d["Cp"].to_numpy()
    S = d["S"].to_numpy()
    phi = d["phi"].to_numpy()
    Hrel = d["Hrel"].to_numpy()

    return JanafPhase(
        name=phase_name,
        Tmin=float(T.min()),
        Tmax=float(T.max()),
        Cp=make_linear_interp(T, Cp),
        S=make_linear_interp(T, S),
        phi=make_linear_interp(T, phi),
        Hrel=make_linear_interp(T, Hrel),
    )


# ----------------------------
# 4) Phase stability comparison + crossing
# ----------------------------

def find_crossing_T(phaseA: JanafPhase, phaseB: JanafPhase,
                    Tlo: float, Thi: float,
                    H_Tr: float = 0.0,
                    ngrid: int = 2001) -> Optional[float]:
    """
    Find approximate temperature where G_A(T) - G_B(T) changes sign.
    Returns None if no sign change in [Tlo, Thi].
    """
    T = np.linspace(Tlo, Thi, ngrid)
    dG = phaseA.G(T, H_Tr=H_Tr) - phaseB.G(T, H_Tr=H_Tr)

    s = np.sign(dG)
    idx = np.where(s[:-1] * s[1:] < 0)[0]
    if len(idx) == 0:
        return None

    i = idx[0]
    # linear root in the bracket [T[i], T[i+1]]
    T1, T2 = T[i], T[i+1]
    y1, y2 = dG[i], dG[i+1]
    return T1 - y1 * (T2 - T1) / (y2 - y1)


# ----------------------------
# 5) Example usage (paste your raw table into RAW)
# ----------------------------


df = parse_janaf_text(RAW)


ph = split_phases_si(df, Tm=1685.0, Tb=3504.616)

solid = build_phase_object("solid", ph["solid"])
liquid = build_phase_object("liquid", ph["liquid"])
gas = build_phase_object("gas", ph["gas"])

# Phase boundary checks (should be near the transition markers)
T_melt = find_crossing_T(solid, liquid, Tlo=1200.0, Thi=2000.0)
T_boil = find_crossing_T(liquid, gas, Tlo=3000.0, Thi=4500.0)

print("Estimated solid-liquid crossing T (K):", T_melt)
print("Estimated liquid-gas crossing T (K):", T_boil)

# Compute G curves (kJ/mol)
T_test = np.array([300.0, 1000.0, 1685.0, 2000.0, 4000.0])
print("G_solid(kJ/mol) at T:", solid.G(T_test[T_test <= solid.Tmax]))
print("G_liquid(kJ/mol) at T:", liquid.G(T_test[(T_test >= liquid.Tmin) & (T_test <= liquid.Tmax)]))
print("G_gas(kJ/mol) at T:", gas.G(T_test[T_test >= gas.Tmin]))


ValueError: zero-size array to reduction operation minimum which has no identity

In [2]:
df

,T,Cp,S,phi,Hrel,note
0,0.00,0.000,0.000,-3.218,0.000,INFINITE
1,100.00,7.268,3.833,33.351,-2.952,
2,200.00,15.636,11.665,20.531,-1.773,
3,298.15,20.000,18.820,18.820,0.000,
4,300.00,20.050,18.943,18.820,0.037,
...,...,...,...,...,...,...
61,5600.00,23.091,232.026,130.489,568.609,
62,5700.00,23.098,232.435,132.274,570.918,
63,5800.00,23.108,232.837,134.005,573.228,
64,5900.00,23.121,233.232,135.683,575.539,


In [3]:
ph

{'solid':           T      Cp       S     phi    Hrel      note
 0      0.00   0.000   0.000  -3.218   0.000  INFINITE
 1    100.00   7.268   3.833  33.351  -2.952          
 2    200.00  15.636  11.665  20.531  -1.773          
 3    298.15  20.000  18.820  18.820   0.000          
 4    300.00  20.050  18.943  18.820   0.037          
 5    400.00  22.142  25.032  19.636   2.159          
 6    500.00  23.330  30.110  21.237   4.436          
 7    600.00  24.154  34.440  23.086   6.812          
 8    700.00  24.803  38.212  24.983   9.260          
 9    800.00  25.359  41.562  26.850  11.769          
 10   900.00  25.874  44.579  28.655  14.331          
 11  1000.00  26.338  47.329  30.387  16.942          
 12  1100.00  26.778  49.860  32.044  19.598          
 13  1200.00  27.196  52.208  33.627  22.297          
 14  1300.00  27.614  54.401  35.142  25.037          
 15  1400.00  28.033  56.463  36.592  27.820          
 16  1500.00  28.451  58.411  37.982  30.644          
 

In [5]:
for k,v in ph.items():
    print(k)
    print(v)

solid
          T      Cp       S     phi    Hrel      note
0      0.00   0.000   0.000  -3.218   0.000  INFINITE
1    100.00   7.268   3.833  33.351  -2.952          
2    200.00  15.636  11.665  20.531  -1.773          
3    298.15  20.000  18.820  18.820   0.000          
4    300.00  20.050  18.943  18.820   0.037          
5    400.00  22.142  25.032  19.636   2.159          
6    500.00  23.330  30.110  21.237   4.436          
7    600.00  24.154  34.440  23.086   6.812          
8    700.00  24.803  38.212  24.983   9.260          
9    800.00  25.359  41.562  26.850  11.769          
10   900.00  25.874  44.579  28.655  14.331          
11  1000.00  26.338  47.329  30.387  16.942          
12  1100.00  26.778  49.860  32.044  19.598          
13  1200.00  27.196  52.208  33.627  22.297          
14  1300.00  27.614  54.401  35.142  25.037          
15  1400.00  28.033  56.463  36.592  27.820          
16  1500.00  28.451  58.411  37.982  30.644          
17  1600.00  28.870  6

In [15]:
def parse_janaf_text(raw: str) -> pd.DataFrame:
    rows = []
    lines = [line.strip() for line in raw.splitlines()]
    for i in range(len(lines)):
        print(i,lines[i])
    
    

parse_janaf_text(raw=RAW)

0 
1 Silicon (Si)	Si1(ref)
2 T(K)	Cp	S	-[G-H(Tr)]/T	H-H(Tr)	delta-f H	delta-f G	log Kf
3 0	0.	0.	INFINITE	-3.218	0.	0.	0.
4 100	7.268	3.833	33.351	-2.952	0.	0.	0.
5 200	15.636	11.665	20.531	-1.773	0.	0.	0.
6 298.15	20.000	18.820	18.820	0.	0.	0.	0.
7 300	20.050	18.943	18.820	0.037	0.	0.	0.
8 400	22.142	25.032	19.636	2.159	0.	0.	0.
9 500	23.330	30.110	21.237	4.436	0.	0.	0.
10 600	24.154	34.440	23.086	6.812	0.	0.	0.
11 700	24.803	38.212	24.983	9.260	0.	0.	0.
12 800	25.359	41.562	26.850	11.769	0.	0.	0.
13 900	25.874	44.579	28.655	14.331	0.	0.	0.
14 1000	26.338	47.329	30.387	16.942	0.	0.	0.
15 1100	26.778	49.860	32.044	19.598	0.	0.	0.
16 1200	27.196	52.208	33.627	22.297	0.	0.	0.
17 1300	27.614	54.401	35.142	25.037	0.	0.	0.
18 1400	28.033	56.463	36.592	27.820	0.	0.	0.
19 1500	28.451	58.411	37.982	30.644	0.	0.	0.
20 1600	28.870	60.261	39.317	33.510	0.	0.	0.
21 1685.000	29.225	61.765	40.412	35.979	CRYSTAL <--> LIQUID
22 1685.000	27.196	91.562	40.412	86.187	TRANSITION
23 1700	27.196	91.803	40.8